In [1]:
%load_ext autoreload
%autoreload 2

import os
import torch
import numpy as np
from glob import glob
from tqdm.notebook import tqdm
from sklearn.linear_model import LogisticRegression
import pandas as pd
from utils.store_data import ReadData
import numpy as np
import torch
from utils.arguments import Arguments
from tqdm import tqdm
from sklearn.decomposition import PCA
from linear_probe import get_linear_acc
from utils.seed import fix_seed
import random
import os

In [2]:
fix_seed(0)
data_reader=ReadData('/root/autodl-tmp/hallscope/DocVQA/train')
data=data_reader.read_all()
random.shuffle(data)

In [3]:
df=pd.DataFrame(data)

In [4]:
df.head()

,img_path,question,answer,most_likely,responses
0,[/root/autodl-tmp/train/documents/jzbn0226_36....,[Which age group foes the IUR definition targe...,"[[targets children age 14 and younger], [child...",{'embedding': [[[ 0.00540161 -0.00043488 0.00...,None
1,[/root/autodl-tmp/train/documents/jpyv0228_1.png],[When is the memorandum dated?\nAnswer the que...,[[11/22/76]],{'embedding': [[[-0.01867676 0.00515747 0.00...,None
2,[/root/autodl-tmp/train/documents/qtjf0226_8.png],[What is the last point in the table?\nAnswer ...,"[[Evaluate Impact on Tak-559], [EVALUATE IMPAC...",{'embedding': [[[ 1.8386841e-03 -1.4724731e-03...,None
3,[/root/autodl-tmp/train/documents/lxjh0227_9.png],[What is the heading of the document?\nAnswer ...,"[[The Great Western Sugar Company], [THE GREAT...",{'embedding': [[[-0.01391602 0.02233887 0.00...,None
4,[/root/autodl-tmp/train/documents/kmhh0024_2.png],[Who are the authors\nAnswer the question usin...,"[[henry l. chung lisa s. winkler], [Henry ...",{'embedding': [[[-0.01483154 -0.00866699 0.00...,None


In [5]:
df.shape

(8347, 5)

In [6]:
i=0

In [7]:
df.iloc[i]['img_path']

['/root/autodl-tmp/train/documents/jzbn0226_36.png']

In [8]:
df.iloc[i]['most_likely']['response']

['children age 14 and younger']

In [9]:
df.iloc[i]['question']

['Which age group foes the IUR definition target?\nAnswer the question using a single word or digit number.\n']

In [10]:
df.iloc[i]['answer']

[['targets children age 14 and younger'],
 ['children age 14 and younger'],
 ['age 14 and younger']]

In [11]:
embeddings=df['most_likely'].apply(lambda x: x['embedding']).to_list()

In [12]:
embeddings=np.concatenate(embeddings,axis=0)

In [13]:
embeddings.shape

(8347, 29, 3584)

In [14]:
def transpose_lines(x):
    f=pd.DataFrame(x)
    transposed=f.apply(pd.Series).T.values.tolist()
    return transposed

df['answer']=df['answer'].apply(transpose_lines)

In [15]:
gt_label=df['answer'].to_list()
gts=[]
for x in gt_label:
    gts+=x
gt_label=gts


In [16]:
responses=df['most_likely'].apply(lambda x: x['response']).to_list()
rsps=[]
for x in responses:
    rsps+=x

In [17]:
i=500
print(rsps[i])
print(gt_label[i])

781.60
['781.60']


In [18]:
from answer_judge.vqaEval import VQAEval

judge=VQAEval(gt_label,rsps)
judge.evaluate()
print(judge.evalQA)
print(judge.accuracy['overall'])

computing accuracy


100%|██████████| 8347/8347 [00:02<00:00, 3490.87it/s]

Done computing accuracy
{0: 33.33, 1: 33.33, 2: 0.0, 3: 66.67, 4: 66.67, 5: 66.67, 6: 66.67, 7: 33.33, 8: 33.33, 9: 33.33, 10: 33.33, 11: 33.33, 12: 33.33, 13: 33.33, 14: 33.33, 15: 33.33, 16: 66.67, 17: 0.0, 18: 33.33, 19: 33.33, 20: 33.33, 21: 33.33, 22: 66.67, 23: 66.67, 24: 33.33, 25: 33.33, 26: 33.33, 27: 66.67, 28: 66.67, 29: 33.33, 30: 33.33, 31: 33.33, 32: 33.33, 33: 33.33, 34: 66.67, 35: 33.33, 36: 66.67, 37: 33.33, 38: 33.33, 39: 0.0, 40: 66.67, 41: 66.67, 42: 33.33, 43: 66.67, 44: 33.33, 45: 0.0, 46: 33.33, 47: 33.33, 48: 33.33, 49: 33.33, 50: 33.33, 51: 33.33, 52: 66.67, 53: 33.33, 54: 66.67, 55: 0.0, 56: 33.33, 57: 0.0, 58: 33.33, 59: 33.33, 60: 0.0, 61: 66.67, 62: 0.0, 63: 33.33, 64: 33.33, 65: 33.33, 66: 33.33, 67: 33.33, 68: 33.33, 69: 66.67, 70: 33.33, 71: 33.33, 72: 33.33, 73: 66.67, 74: 33.33, 75: 33.33, 76: 0.0, 77: 33.33, 78: 66.67, 79: 33.33, 80: 33.33, 81: 33.33, 82: 33.33, 83: 33.33, 84: 0.0, 85: 0.0, 86: 33.33, 87: 66.67, 88: 0.0, 89: 33.33, 90: 33.33, 91: 33.3

In [19]:
gt_scores=[judge.evalQA[i] for i in range(len(judge.evalQA))]

In [20]:
embeddings=df['most_likely'].apply(lambda x: x['embedding']).to_list()
embeddings=np.concatenate(embeddings,axis=0)
embeddings.shape

(8347, 29, 3584)

In [21]:
is_correct=[]
threshold=30
for i,x in enumerate(gt_scores):
    if x >=threshold:
        is_correct.append(1)
    else:
        is_correct.append(0)
is_correct=np.array(is_correct)

In [22]:
print(f'There are {len(is_correct)} responses. {sum(is_correct==1)} answer correctly and {sum(is_correct==0)} answer wrongly')

There are 8347 responses. 7806 answer correctly and 541 answer wrongly


In [64]:
length=len(is_correct)
train_rate=0.8

all_indices = np.random.permutation(length)
train_val_idxs = all_indices[:int(
    train_rate * length)]  # trainset and validation index

test_idxs=all_indices[int(
    train_rate * length):]  #test index

validset_len = 2000
# exclude validation samples.
train_idxs = train_val_idxs[:- validset_len]  # trainset index
val_idxs = train_val_idxs[- validset_len:]  # validation index
hall_label_test = []
hall_label_wild = []
hall_label_val = []

'''get testset, wildset and valset. The valset is used for determining the hype-parameters'''
hall_label_test = is_correct[test_idxs]
hall_label_wild = is_correct[train_idxs]
hall_label_val = is_correct[val_idxs]

print(f'trainset length:{len(hall_label_wild)}')
print(f'validset length:{len(hall_label_val)}')
print(f'testset length:{len(hall_label_test)}')
print(f'trainset postive: {sum(hall_label_wild==1)} negative: {sum(hall_label_wild==0)}')
print(f'valset postive: {sum(hall_label_val==1)} negative: {sum(hall_label_val==0)}')
print(
    f'testset postive: {sum(hall_label_test==1)} negative: {sum(hall_label_test==0)}')

trainset length:4677
validset length:2000
testset length:1670
trainset postive: 4373 negative: 304
valset postive: 1877 negative: 123
testset postive: 1556 negative: 114


In [65]:
all_embeddings = embeddings
all_embeddings.shape

(8347, 29, 3584)

In [66]:
embedding_train = all_embeddings[train_idxs]
embedding_val = all_embeddings[val_idxs]
embedding_test = all_embeddings[test_idxs]

In [67]:
%load_ext autoreload
%autoreload 2
from ml_utils.grid_search import GridSearch
from ml_utils.PCA_discriminator import PCALinear,PCAKernel,KernelPCA
from sklearn.metrics import roc_auc_score,accuracy_score,roc_curve,f1_score,balanced_accuracy_score
from ml_utils.metrics import auc_pr

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Grid Search for Best 'n_components' & 'layer'

In [68]:
embedding_val.shape

(2000, 29, 3584)

In [69]:
M=3584
def evaluator(n_components,i_layer,X,y):
    discriminator=KernelPCA(X[:,i_layer,:],n_components,M,method='CoRP',gamma=1)
    scores=discriminator.get_score(X[:,i_layer,:])
    
    return max(roc_auc_score(y,scores),roc_auc_score(y,-scores))
    #split=discriminator.get_best_split(scores,y)
    #y_pred=scores>split
    #return accuracy_score(y,y_pred)
    #return balanced_accuracy_score(y,y_pred)
    #
    #split=discriminator.get_best_split(y)
    #y_preds=(scores>split)
    #return accuracy_score(y,y_preds)

In [70]:
# graid search for best hyper-parameters on validation set
grid={
    'n_components':range(5,1,-1),
    'layer':range(0,embedding_val.shape[1])
}
grid_search=GridSearch(evaluator,grid,embedding_val,hall_label_val)
best_paras=grid_search.search()

Grid Searching for best n_components,layer


  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/29 [00:00<?, ?it/s]

found best n_components:5,layer:0 Score: 0.5242776268998705
found best n_components:5,layer:2 Score: 0.5523084319814962


  0%|          | 0/29 [00:00<?, ?it/s]

found best n_components:4,layer:0 Score: 0.5747776897055065


  0%|          | 0/29 [00:00<?, ?it/s]

  0%|          | 0/29 [00:00<?, ?it/s]

In [71]:
best_paras

{'n_components': 4, 'layer': 0, 'best_score': np.float64(0.5747776897055065)}

In [72]:
best_layer=best_paras['layer']

View Results on Training set and Test set

In [73]:
discriminator=KernelPCA(embedding_val[:,best_layer,:],best_paras['n_components'],M,method='CoRP',gamma=1)
scores=discriminator.get_score(embedding_val[:,best_layer,:])
best_split=discriminator.get_best_split(scores,hall_label_val)

In [74]:
discriminator=KernelPCA(embedding_train[:,best_layer,:],best_paras['n_components'],M,method='CoRP',gamma=1)
scores=discriminator.get_score(embedding_train[:,best_layer,:])
y_train=(scores>best_split)
print(f'train set acc: {accuracy_score(hall_label_wild,y_train)} AUROC:{roc_auc_score(hall_label_wild,scores)}')

train set acc: 0.8997220440453282 AUROC:0.5239372585362331


In [75]:
discriminator=KernelPCA(embedding_test[:,best_layer,:],best_paras['n_components'],M,method='CoRP',gamma=1)
scores=discriminator.get_score(embedding_test[:,best_layer,:])
preds=(scores>best_split)
print(f'test set acc: {accuracy_score(hall_label_test,preds)} AUROC:{roc_auc_score(hall_label_test,scores)}')

test set acc: 0.8880239520958084 AUROC:0.47422259053804183


Train Linear Prob to Get Better Results

In [76]:
X_train=embedding_train[:, best_layer,:]
y_train=y_train

X_test=embedding_test[:, best_layer,:]
y_test=hall_label_test

In [77]:
best_acc, final_acc, (
            clf, best_state, best_preds, preds, labels_val), losses_train = get_linear_acc(
            X_train,
            y_train,
           X_train,
            y_train,
            2, epochs=50,
            print_ret=True,
            batch_size=512,
            cosine=True,
            nonlinear=True,
            learning_rate=0.05,
            weight_decay=0.0003)

In [78]:
clf.eval()
output = clf(torch.from_numpy(X_test).to(torch.float32).cuda())
scores_test = torch.sigmoid(output).cpu().data.numpy()
pred_test=scores_test>0.5
print(f'test set acc: {accuracy_score(y_test,pred_test)} f1:{f1_score(y_test,pred_test)} auroc:{roc_auc_score(y_test,scores_test)}  b-acc:{balanced_accuracy_score(y_test,pred_test)} auc-pr:{auc_pr(y_test,scores_test)}')

test set acc: 0.9317365269461078 f1:0.9646621202727836 auroc:0.4589365444459478  b-acc:0.5 auc-pr:0.9274012268736966
